Purpose: The purpose of this notebook is to perform k-means clustering of 58 NSCLC tumors using k=2. This analysis uses the strategy of training a k-means clustering model using half of the dataset, and then testing the model on the other half of the dataset. Accuracy scores are calculated for the model's predictions on both datasets. 

Author: Maya Bose

Date: Fall, 2021

In [1]:
import pandas as pd
import GEOparse
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import numpy as np
np.random.seed(2021)

In [2]:
# Replace this with Miguel's annotated dataframe
df = pd.read_csv("/home/mbose/DropboxUMich/Coursework/BIOINFO575/project/LimBoYuMoo/rawDataFrameGSE10245.csv")
# Transpose the dataframe. We want the sample labels in 1 column
df = df.T
df.columns = df.iloc[0]
df = df[1:]

rawDataGSE10245 = GEOparse.get_GEO(filepath="/home/mbose/DropboxUMich/Coursework/BIOINFO575/project/LimBoYuMoo/GSE10245_family.soft.gz", silent=True)
tumor_subtypes = list()
for sample in rawDataGSE10245.gsms:
    tumor_subtype = rawDataGSE10245.gsms.get(sample).metadata['characteristics_ch1'][0].split(":")[1].strip()
    if tumor_subtype == "adenocarcinoma":
        tumor_subtypes.append("AD")
    elif tumor_subtype == "squamous cell carcinoma":
        tumor_subtypes.append("SCC")
    else:
        tumor_subtypes.append("UNKNOWN")
df = df.assign(tumor_subtype = tumor_subtypes)
df.head()

/home/mbose/miniconda3/lib/python3.9/site-packages/GEOparse/GEOparse.py:545: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  table_data = parse_table_data(gpl_soft)


ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioB-3_at,AFFX-r2-Ec-bioB-5_at,AFFX-r2-Ec-bioB-M_at,AFFX-r2-Ec-bioC-3_at,AFFX-r2-Ec-bioC-5_at,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,tumor_subtype
GSM258551,9.129905,8.034022,3.56452,4.74649,2.320698,5.519153,3.339182,2.775395,8.303437,2.981327,...,8.857847,9.369037,9.435077,10.936404,10.730983,13.599488,13.031726,15.028729,14.586347,AD
GSM258552,9.843349,7.973332,4.994852,5.197306,2.24852,5.081258,2.934516,2.617097,9.145519,4.619668,...,7.865127,8.116953,8.306683,9.986505,9.719266,12.847711,12.250033,14.440756,14.072366,AD
GSM258553,9.730661,8.834045,5.066018,5.234618,2.259504,4.657257,3.007192,2.634559,9.159054,2.746464,...,6.996104,7.803869,8.029131,9.468096,9.101115,12.384142,11.798363,14.439887,14.011392,SCC
GSM258554,9.032165,7.723965,4.95858,6.07818,2.262787,4.535683,3.167112,3.127495,7.770354,6.212399,...,8.221659,8.43409,8.609512,10.064087,9.763076,12.969199,12.307684,14.557363,14.162145,AD
GSM258555,10.281793,9.0408,4.951835,5.205632,2.207531,3.731919,2.71179,2.707079,6.380928,4.528499,...,8.042156,8.174685,8.620098,10.005558,9.651785,12.924465,12.243207,14.612223,14.122751,SCC


Lets partition our df into AD and SCC samples. randomly choose 20 AD samples and 9 SCC sample. We will designate these the "test" set. The remainder of the samples will be the "validation" set. 

In [3]:
ad = df[df["tumor_subtype"] == "AD"]
scc = df[df["tumor_subtype"] == "SCC"]
print(f"There are {ad.shape[0]} AD samples and {scc.shape[0]} SCC samples")

There are 40 AD samples and 18 SCC samples


We want to use half of the AD samples as our "training" set, and the other half as our "testing" set. Ditto for the SCC samples. We can use the sklearn.model_selection train_test_split function to do this. test_size indicates the ratio of training samples to testing samples; in this case, the ratio is 50:50 (half).

In [4]:
train_ad, test_ad = train_test_split(ad, test_size = 0.5)
train_scc, test_scc = train_test_split(scc, test_size = 0.5)

Let's combine our "training" set for AD samples with our "training" set for SCC samples into a single training set. Do the same to create a single testing set. We also want to remove the tumor subtype annotations, so that we can then use the model to predict these annotations, but we will also save the labels in a seperate variable for testing the accuracy of the model later.

In [5]:
# Combine AD training samples with SCC training samples to create a single training DF
train = pd.concat([train_ad, train_scc])
# Save the tumor subtype labels for accuracy testing later
train_labels = train["tumor_subtype"].values
# drop the tumor subtype labels to use the DF in clustering
train = train.drop("tumor_subtype", axis = 1)

# Combine AD testing samples with SCC testing samples to create a single testing DF
test = pd.concat([test_ad, test_scc])
# Save the tumor subtype labels for accuracy testing later
test_labels = test["tumor_subtype"].values
# drop the tumor subtype labels to use the DF in clustering
test = test.drop("tumor_subtype", axis = 1)

In [6]:
# Check out what one of these DFs looks like to make sure everything's good
train.head()

ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Bs-thr-M_s_at,AFFX-r2-Ec-bioB-3_at,AFFX-r2-Ec-bioB-5_at,AFFX-r2-Ec-bioB-M_at,AFFX-r2-Ec-bioC-3_at,AFFX-r2-Ec-bioC-5_at,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at
GSM258600,10.130757,8.273698,4.801579,4.807654,2.232021,6.251143,2.95905,2.606255,9.747887,2.731335,...,8.520138,8.540452,8.897361,9.071301,10.624876,10.212906,13.323579,12.834769,14.779129,14.471579
GSM258574,8.914079,8.23263,6.634602,5.086281,2.212639,5.626913,2.949485,2.578616,9.465973,4.645915,...,9.503383,7.862734,8.362031,8.540746,10.206568,9.676948,13.081193,12.53171,14.738462,14.298989
GSM258589,10.453398,8.4611,5.017366,5.001853,2.228587,6.037936,2.958407,2.588412,7.611752,2.657426,...,9.163212,8.246976,8.709728,8.75595,10.20847,10.004794,13.210599,12.646422,14.782507,14.325541
GSM258564,8.579661,8.207932,4.681478,5.231288,2.239315,4.543166,3.277536,2.612152,5.775388,2.970464,...,9.37183,8.173794,8.519084,8.636633,10.271777,9.882191,13.125126,12.484588,14.694147,14.306079
GSM258560,10.643545,9.14226,5.368356,5.19582,2.246563,4.174514,2.968054,2.609029,7.356981,2.722873,...,8.951526,8.004727,8.110386,8.529225,9.872159,9.457238,12.767953,12.280146,14.523177,14.159513


Now, we can train the model using the training set

In [7]:
# Initialize KMeans object
kmeans = KMeans(n_clusters = 2, )
# Train the model
kmeans.fit(train)

# Use the trained model to predict tumor states in the training set
train_kmeans = kmeans.predict(train)
# Use the trained model to predict tumor states in the testing set
test_kmeans = kmeans.predict(test)

In [8]:
# Check out what the clusters look like
test_kmeans

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1], dtype=int32)

We will map the 0 cluster to SCC and the 1 cluster to AD.

In [9]:
train_kmeans = np.where(train_kmeans == 0, "SCC", "AD")
test_kmeans = np.where(test_kmeans == 0, "SCC", "AD")

Neat. Step 5 completed! Now for step 6 (computing and displaying the accuracy of the model for the training and testing data). We can do this by using sklearn.metrics functions accuracy_score to compare the known labels (stored in variables earlier) to the cluster predictions. 

Accuracy score for the training data set:

In [10]:
print(f"The accuracy score when training on half of the data is {accuracy_score(train_labels, train_kmeans)}.")

The accuracy score when training on half of the data is 0.8275862068965517.


Accuracy score for the testing data set:

In [11]:
print(f"The accuracy score when testing on half of the data is {accuracy_score(test_labels, test_kmeans)}.")

The accuracy score when testing on half of the data is 0.9310344827586207.


Step 6 complete!